<a href="https://colab.research.google.com/github/JawadSk12/ML-Lab/blob/main/ML_LAB_COMPETITION_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit plotly seaborn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.9 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.1
    Uninstalling cachetools-7.0.1:
      Successfully uninstalled cachetools-7.0.1


In [2]:
# Install dependencies as needed:
!pip install kagglehub[pandas-datasets]
!pip install -q --upgrade kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 6.5 MB/s eta 0:00:00


In [3]:

import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "athlete_events.csv"

df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "heesoo37/120-years-of-olympic-history-athletes-and-results",
    "athlete_events.csv"
)
'''
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "heesoo37/120-years-of-olympic-history-athletes-and-results",
  file_path
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)
'''

print("First 5 records:", df.head())

100%|██████████| 5.43M/5.43M [00:00<00:00, 71.3MB/s]

Extracting zip of athlete_events.csv...


First 5 records:    ID                      Name Sex   Age  Height  Weight            Team  \
0   1                 A Dijiang   M  24.0   180.0    80.0           China   
1   2                  A Lamusi   M  23.0   170.0    60.0           China   
2   3       Gunnar Nielsen Aaby   M  24.0     NaN     NaN         Denmark   
3   4      Edgar Lindenau Aabye   M  34.0     NaN     NaN  Denmark/Sweden   
4   5  Christine Jacoba Aaftink   F  21.0   185.0    82.0     Netherlands   

   NOC        Games  Year  Season       City          Sport  \
0  CHN  1992 Summer  1992  Summer  Barcelona     Basketball   
1  CHN  2012 Summer  2012  Summer     London           Judo   
2  DEN  1920 Summer  1920  Summer  Antwerpen       Football   
3  DEN  1900 Summer  1900  Summer      Paris     Tug-Of-War   
4  NED  1988 Winter  1988  Winter    Calgary  Speed Skating   

                              Event Medal  
0       Basketball Men's Basketball   NaN  
1      Judo Men's Extra-Lightweight   NaN  
2        

In [4]:
%%writefile app.py


import streamlit as st
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff

import kagglehub
from kagglehub import KaggleDatasetAdapter
import preprocessor, helper


@st.cache_data
def load_data():
    df = kagglehub.dataset_load(
        KaggleDatasetAdapter.PANDAS,
        "heesoo37/120-years-of-olympic-history-athletes-and-results",
        "athlete_events.csv"
    )

    region_df = kagglehub.dataset_load(
        KaggleDatasetAdapter.PANDAS,
        "heesoo37/120-years-of-olympic-history-athletes-and-results",
        "noc_regions.csv"
    )

    df_clean = preprocessor.preprocess(df, region_df)
    return df_clean

df = load_data()

st.sidebar.title("🏅 Olympics Analysis Dashboard")
st.sidebar.image(
    'https://e7.pngegg.com/pngimages/1020/402/png-clipart-2024-summer-olympics-brand-circle-area-olympic-rings-olympics-logo-text-sport.png'
)

user_menu = st.sidebar.radio(
    'Select an Option',
    ('Medal Tally', 'Overall Analysis', 'Country-wise Analysis', 'Athlete wise Analysis')
)



if user_menu == 'Medal Tally':
    st.sidebar.header("Medal Tally Filters")
    years, country = helper.country_year_list(df)
    selected_year = st.sidebar.selectbox("Select Year", years)
    selected_country = st.sidebar.selectbox("Select Country", country)

    medal_tally = helper.fetch_medal_tally(df, selected_year, selected_country)

    # Dynamic title
    if selected_year == 'Overall' and selected_country == 'Overall':
        st.title("🏆 Overall Medal Tally")
    elif selected_year != 'Overall' and selected_country == 'Overall':
        st.title(f"🏆 Medal Tally - {selected_year}")
    elif selected_year == 'Overall' and selected_country != 'Overall':
        st.title(f"🏆 {selected_country} Overall Performance")
    else:
        st.title(f"🏆 {selected_country} in {selected_year}")

    st.table(medal_tally)



elif user_menu == 'Overall Analysis':
    editions = df['Year'].nunique()
    cities = df['City'].nunique()
    sports = df['Sport'].nunique()
    events = df['Event'].nunique()
    athletes = df['Name'].nunique()
    nations = df['region'].nunique()

    st.title("📊 Top Statistics")
    col1, col2, col3 = st.columns(3)
    col1.metric("Editions", editions)
    col2.metric("Hosts", cities)
    col3.metric("Sports", sports)
    col1, col2, col3 = st.columns(3)
    col1.metric("Events", events)
    col2.metric("Nations", nations)
    col3.metric("Athletes", athletes)

    # Nations over time
    nations_over_time = helper.data_over_time(df, 'region')
    fig = px.line(nations_over_time, x="Edition", y="region")
    st.title("🌍 Participating Nations Over Time")
    st.plotly_chart(fig)

    # Events over time
    events_over_time = helper.data_over_time(df, 'Event')
    fig = px.line(events_over_time, x="Edition", y="Event")
    st.title("🎯 Events Over Time")
    st.plotly_chart(fig)

    # Athletes over time
    athlete_over_time = helper.data_over_time(df, 'Name')
    fig = px.line(athlete_over_time, x="Edition", y="Name")
    st.title("👥 Athletes Over Time")
    st.plotly_chart(fig)

    # Heatmap Sport vs Year
    st.title("🔥 Number of Events (Sport vs Year)")
    fig, ax = plt.subplots(figsize=(20, 20))
    x = df.drop_duplicates(['Year', 'Sport', 'Event'])
    heatmap_data = x.pivot_table(
        index='Sport',
        columns='Year',
        values='Event',
        aggfunc='count'
    ).fillna(0)
    sns.heatmap(heatmap_data, annot=True)
    st.pyplot(fig)

    st.title("🏅 Most Successful Athletes")
    sport_list = df['Sport'].unique().tolist()
    sport_list.sort()
    sport_list.insert(0, 'Overall')
    selected_sport = st.selectbox('Select a Sport', sport_list)
    x = helper.most_successful(df, selected_sport)
    st.table(x)



elif user_menu == 'Country-wise Analysis':
    st.sidebar.title('Country Filters')
    country_list = df['region'].dropna().unique().tolist()
    country_list.sort()
    selected_country = st.sidebar.selectbox('Select a Country', country_list)

    country_df = helper.yearwise_medal_tally(df, selected_country)
    fig = px.line(country_df, x="Year", y="Medal")
    st.title(f"📈 {selected_country} Medal Tally Over Years")
    st.plotly_chart(fig)

    st.title(f"🔥 {selected_country} Performance by Sport")
    pt = helper.country_event_heatmap(df, selected_country)
    fig, ax = plt.subplots(figsize=(20, 20))
    sns.heatmap(pt, annot=True)
    st.pyplot(fig)

    st.title(f"🏅 Top 10 Athletes of {selected_country}")
    top10_df = helper.most_successful_countrywise(df, selected_country)
    st.table(top10_df)



elif user_menu == 'Athlete wise Analysis':
    athlete_df = df.drop_duplicates(subset=['Name', 'region'])

    # Age distribution
    x1 = athlete_df['Age'].dropna()
    x2 = athlete_df[athlete_df['Medal'] == 'Gold']['Age'].dropna()
    x3 = athlete_df[athlete_df['Medal'] == 'Silver']['Age'].dropna()
    x4 = athlete_df[athlete_df['Medal'] == 'Bronze']['Age'].dropna()

    fig = ff.create_distplot(
        [x1, x2, x3, x4],
        ['Overall Age', 'Gold', 'Silver', 'Bronze'],
        show_hist=False,
        show_rug=False
    )
    fig.update_layout(width=1000, height=600)
    st.title("📊 Age Distribution of Athletes")
    st.plotly_chart(fig)

    # Height vs Weight
    st.title("⚖ Height vs Weight")
    sport_list = df['Sport'].unique().tolist()
    sport_list.sort()
    sport_list.insert(0, 'Overall')
    selected_sport = st.selectbox('Select Sport', sport_list)
    temp_df = helper.weight_v_height(df, selected_sport)

    fig, ax = plt.subplots()
    sns.scatterplot(
        data=temp_df,
        x='Weight',
        y='Height',
        hue='Medal',
        style='Sex',
        s=60
    )
    st.pyplot(fig)

    st.title("👨‍👩‍👧‍👦 Men vs Women Participation")
    final = helper.men_vs_women(df)
    fig = px.line(final, x="Year", y=["Male", "Female"])
    fig.update_layout(width=1000, height=600)
    st.plotly_chart(fig)



Writing app.py


In [5]:
%%writefile preprocessor.py

import pandas as pd

def preprocess(df, region_df):

    df = df[df['Season'] == 'Summer']

    df = df.merge(region_df, on='NOC', how='left')

    df.drop_duplicates(inplace=True)

    df = pd.concat([df, pd.get_dummies(df['Medal'])], axis=1)

    return df


Writing preprocessor.py


In [6]:
%%writefile helper.py

import numpy as np

# ==============
# 1️⃣ FETCH MEDAL TALLY
# Returns a medal tally table based on year and country selection
# ==============
def fetch_medal_tally(df, year, country):

    medal_df = df.drop_duplicates(
        subset=['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal']
    )

    flag = 0

    if year == 'Overall' and country == 'Overall':
        temp_df = medal_df

    if year == 'Overall' and country != 'Overall':
        flag = 1
        temp_df = medal_df[medal_df['region'] == country]

    if year != 'Overall' and country == 'Overall':
        temp_df = medal_df[medal_df['Year'] == int(year)]

    if year != 'Overall' and country != 'Overall':
        temp_df = medal_df[
            (medal_df['Year'] == int(year)) & (medal_df['region'] == country)
        ]

    if flag == 1:
        x = temp_df.groupby('Year').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Year').reset_index()
    else:
        x = temp_df.groupby('region').sum()[['Gold', 'Silver', 'Bronze']].sort_values('Gold', ascending=False).reset_index()

    x['total'] = x['Gold'] + x['Silver'] + x['Bronze']

    x['Gold'] = x['Gold'].astype('int')
    x['Silver'] = x['Silver'].astype('int')
    x['Bronze'] = x['Bronze'].astype('int')
    x['total'] = x['total'].astype('int')

    return x


# ==============
# 2️⃣ DROPDOWN OPTIONS: YEARS & COUNTRIES
# Returns sorted lists for sidebar dropdowns
# ==============
def country_year_list(df):

    years = df['Year'].unique().tolist()
    years.sort()
    years.insert(0, 'Overall')

    country = np.unique(df['region'].dropna().values).tolist()
    country.sort()
    country.insert(0, 'Overall')

    return years, country


# ==============
# 3️⃣ DATA OVER TIME
# Returns data for line plots showing trends over years
# col = column name (e.g., 'region', 'Event', 'Name')
# ==============
def data_over_time(df, col):

    nations_over_time = df.drop_duplicates(['Year', col])['Year'].value_counts().reset_index().sort_values('index')

    nations_over_time.rename(columns={'index': 'Edition', 'Year': col}, inplace=True)

    return nations_over_time


# ==============
# 4️⃣ MOST SUCCESSFUL ATHLETES
# Returns top 15 athletes for a sport or overall
# ==============
def most_successful(df, sport):

    temp_df = df.dropna(subset=['Medal'])

    if sport != 'Overall':
        temp_df = temp_df[temp_df['Sport'] == sport]

    x = temp_df['Name'].value_counts().reset_index().head(15).merge(
        df, left_on='index', right_on='Name', how='left'
    )[['index', 'Name_x', 'Sport', 'region']].drop_duplicates('index')

    x.rename(columns={'index': 'Name', 'Name_x': 'Medals'}, inplace=True)

    return x


# ==============
# 5️⃣ YEARWISE MEDAL TALLY FOR COUNTRY
# Returns medal counts for a country across years
# ==============
def yearwise_medal_tally(df, country):

    temp_df = df.dropna(subset=['Medal'])

    temp_df.drop_duplicates(subset=['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'], inplace=True)

    new_df = temp_df[temp_df['region'] == country]

    final_df = new_df.groupby('Year').count()['Medal'].reset_index()

    return final_df


# ==============
# 6️⃣ COUNTRY EVENT HEATMAP
# Returns pivot table of Sports vs Years for a country
# ==============
def country_event_heatmap(df, country):

    temp_df = df.dropna(subset=['Medal'])

    temp_df.drop_duplicates(subset=['Team', 'NOC', 'Games', 'Year', 'City', 'Sport', 'Event', 'Medal'], inplace=True)

    new_df = temp_df[temp_df['region'] == country]

    pt = new_df.pivot_table(index='Sport', columns='Year', values='Medal', aggfunc='count').fillna(0)

    return pt


# ==============
# 7️⃣ MOST SUCCESSFUL COUNTRYWISE
# Returns top 10 athletes from a specific country
# ==============
def most_successful_countrywise(df, country):

    temp_df = df.dropna(subset=['Medal'])

    temp_df = temp_df[temp_df['region'] == country]

    x = temp_df['Name'].value_counts().reset_index().head(10).merge(
        df, left_on='index', right_on='Name', how='left'
    )[['index', 'Name_x', 'Sport']].drop_duplicates('index')

    x.rename(columns={'index': 'Name', 'Name_x': 'Medals'}, inplace=True)

    return x


# ==============
# 8️⃣ HEIGHT VS WEIGHT DATA
# Returns athlete data for scatter plot
# ==============
def weight_v_height(df, sport):

    athlete_df = df.drop_duplicates(subset=['Name', 'region'])

    athlete_df['Medal'].fillna('No Medal', inplace=True)

    if sport != 'Overall':
        temp_df = athlete_df[athlete_df['Sport'] == sport]
        return temp_df
    else:
        return athlete_df


# ==============
# 9️⃣ MEN VS WOMEN PARTICIPATION
# Returns yearly counts of male and female athletes
# ==============
def men_vs_women(df):

    athlete_df = df.drop_duplicates(subset=['Name', 'region'])

    men = athlete_df[athlete_df['Sex'] == 'M'].groupby('Year').count()['Name'].reset_index()
    women = athlete_df[athlete_df['Sex'] == 'F'].groupby('Year').count()['Name'].reset_index()

    final = men.merge(women, on='Year', how='left')
    final.rename(columns={'Name_x': 'Male', 'Name_y': 'Female'}, inplace=True)

    final.fillna(0, inplace=True)

    return final


Writing helper.py
